In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan  7 17:46:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              46W / 350W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# %%
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

!git config --global user.email "guiramison@gmail.com"
!git config --global user.name "Ramison"

from datasets import load_dataset
from transformers import AutoTokenizer

# %%
raw_datasets = load_dataset("squad_v1_pt")

# %%
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

# %%
model_checkpoint = "PORTULAN/albertina-900m-portuguese-ptbr-encoder-brwac"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# %%
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# %%
tokenized_train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(tokenized_train_dataset)

# %%
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

# %%
tokenized_validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(tokenized_validation_dataset)

# %%
small_train_dataset = tokenized_train_dataset.shuffle(seed=42) #.select(range(1000))
small_eval_dataset = tokenized_validation_dataset.shuffle(seed=42) #.select(range(1000))

# %%
print("Context: ", small_train_dataset[0])

# %%
tokenized_train_dataset

# %%
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)

eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

# %%
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
trained_checkpoint = "PORTULAN/albertina-900m-portuguese-ptbr-encoder-brwac"
model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

# %%
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

# %%
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Context:  Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.
Question:  A quem a Virgem Maria supostamente apareceu em 1858 em Lourdes, na França?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [0]}


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Context:  {'input_ids': [1, 56015, 14805, 2439, 104, 3462, 71, 4283, 17221, 670, 208, 5691, 24226, 30911, 34497, 1658, 44, 2, 24226, 35747, 53767, 1658, 10, 1559, 13405, 12, 9228, 169, 16250, 93, 13564, 106, 762, 16250, 93, 6510, 10, 67230, 10528, 475, 47122, 12, 3153, 2285, 4, 78, 1559, 13405, 12, 9228, 11167, 18078, 68293, 653, 115835, 23579, 447, 49467, 516, 85251, 52535, 4283, 12304, 104243, 475, 2213, 91050, 72424, 4, 55525, 86336, 5155, 99, 10, 10, 20147, 9555, 104, 297, 71, 1084, 3153, 2285, 4, 24226, 10, 64489, 9555, 52535, 8920, 9181, 3543, 6031, 10291, 12560, 4, 5841, 623, 11487, 46824, 29127, 459, 475, 629, 3056, 4796, 18736, 1230, 97514, 5882, 44180, 43563, 475, 475, 1555, 41, 3292, 6510, 10, 3153, 8055, 4, 55525, 74400, 9466, 1658, 623, 52967, 115, 154, 1658, 16, 4153, 1332, 82, 1049, 6, 11487, 2082, 72388, 39418, 2439, 12, 453, 1312, 2245, 1531, 20002, 27585, 89608, 12, 40229, 17408, 45659, 4, 868, 2082, 72388, 80098, 3569, 73956, 516, 12949, 10463, 33444, 12, 957, 6, 569

Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at PORTULAN/albertina-900m-portuguese-ptbr-encoder-brwac and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Nova seção

In [ ]:
# %%
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# %%

import torch
torch.cuda.empty_cache()

# %%
from tqdm.auto import tqdm
from transformers import DefaultDataCollator

squad_v2 = False
batch_size = 8
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir= "albertina_tcc_2024",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16 = True
)

trainer = Trainer(
    model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train()

# %%
trainer.save_model("albertina_tcc_2024")

Epoch,Training Loss,Validation Loss
1,1.320400,No log
2,0.970500,No log
3,0.599000,No log


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %%
from google.colab import files
import shutil
!mkdir -p /content/drive/MyDrive/albertina_tcc_2024
!cp -av albertina_tcc_2024 /content/drive/MyDrive/albertina_tcc_2024

'albertina_tcc_2024' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024'
'albertina_tcc_2024/runs' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024/runs'
'albertina_tcc_2024/runs/Jan07_17-51-34_fa218379f370' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024/runs/Jan07_17-51-34_fa218379f370'
'albertina_tcc_2024/runs/Jan07_17-51-34_fa218379f370/events.out.tfevents.1704649895.fa218379f370.3301.0' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024/runs/Jan07_17-51-34_fa218379f370/events.out.tfevents.1704649895.fa218379f370.3301.0'
'albertina_tcc_2024/runs/Jan07_18-15-08_fa218379f370' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024/runs/Jan07_18-15-08_fa218379f370'
'albertina_tcc_2024/runs/Jan07_18-15-08_fa218379f370/events.out.tfevents.1704651308.fa218379f370.3301.1' -> '/content/drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024/runs/Jan07_18-15-08_fa218379f370/events.out.tfevents.1704651308.fa218379f370.3301.1'


In [ ]:
# %%
pad_on_right = tokenizer.padding_side == "right"
doc_stride = 128
max_length = 384

# %%
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()


# %%
output.start_logits.shape, output.end_logits.shape

# %%
n_best_size = 20

import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )



# %%
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


# %%
validation_features = raw_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

# %%
print(validation_features)

# %%
raw_predictions = trainer.predict(validation_features)

# %%
print(raw_predictions)

# %%
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

# %%
print(validation_features)

# %%
from tqdm.auto import tqdm
import collections

# %%
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.

            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions



# %%
from datasets import load_metric

final_predictions = postprocess_qa_predictions(raw_datasets["validation"], validation_features, raw_predictions.predictions)

metric = load_metric("squad_v2" if squad_v2 else "squad")

# print(final_predictions)


# %%
# print(raw_predictions.predictions)

# %%
# print(final_predictions.items())

# %%
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]


print(formatted_predictions[0])
print(raw_datasets["validation"]["answers"][0])

references = [{"id": ex["id"], "answers": ex["answers"]} for ex in raw_datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)



# %%
# raw_datasets["validation"]["answers"]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 12126
})


PredictionOutput(predictions=(array([[ 0.88623047, -4.1171875 , -5.4648438 , ..., -7.2539062 ,
        -7.2539062 , -7.2539062 ],
       [ 0.6982422 , -3.8789062 , -5.0585938 , ..., -7.2539062 ,
        -7.2539062 , -7.2539062 ],
       [ 3.4453125 , -2.625     , -5.453125  , ..., -7.2539062 ,
        -7.2539062 , -7.2539062 ],
       ...,
       [ 1.5986328 , -3.5214844 , -5.578125  , ..., -7.2578125 ,
        -7.2578125 , -7.2578125 ],
       [ 0.6665039 , -4.6328125 , -5.578125  , ..., -7.2578125 ,
        -7.2578125 , -7.2578125 ],
       [ 1.9921875 , -4.234375  , -3.6152344 , ..., -7.2578125 ,
        -7.2578125 , -7.2578125 ]], dtype=float32), array([[ -2.2597656 ,  -6.9609375 ,  -8.0859375 , ..., -10.0703125 ,
        -10.0703125 , -10.0703125 ],
       [ -2.3007812 ,  -7.6289062 ,  -8.3984375 , ..., -10.0703125 ,
        -10.0703125 , -10.0703125 ],
       [ -0.28857422,  -5.6757812 ,  -6.0351562 , ..., -10.0703125 ,
        -10.0703125 , -10.0703125 ],
       ...,
       [ -1

  0%|          | 0/10570 [00:00<?, ?it/s]

<ipython-input-8-531369997392>:202: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'id': '56be4db0acb8001400a502ec', 'prediction_text': ' Denver Broncos'}
{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [182, 182, 182]}


{'exact_match': 44.57899716177862, 'f1': 55.709254273800624}

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

# Load the fine-tuned ALBERT model and tokenizer
model_path = 'drive/MyDrive/albertina_tcc_2024/albertina_tcc_2024'
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example question and context
question = "Quais desafios o brasil enfrenta?"
context = """
            Com uma população diversificada composta por uma mistura de povos indígenas, europeus, africanos e asiáticos, o Brasil é conhecido por sua rica herança cultural. A música, a dança e as festividades são elementos essenciais da cultura brasileira, com estilos como samba, bossa nova e frevo ganhando destaque internacionalmente.
            Economicamente, o Brasil é uma das maiores economias emergentes do mundo, com setores diversificados que incluem agricultura, indústria, serviços e recursos naturais. No entanto, o país também enfrenta desafios significativos, como desigualdade social, pobreza, corrupção e questões ambientais, que continuam a ser áreas de preocupação e foco para o desenvolvimento sustentável."""

# Tokenize the question and context
inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)

# Get the answer from the model
with torch.no_grad():
    outputs = model(**inputs)

print(outputs)

start_index = outputs.start_logits.argmax()
end_index = outputs.end_logits.argmax()

# Convert indices to text
predict_answer_tokens = inputs.input_ids[0, start_index : end_index + 1]
tokenizer.decode(predict_answer_tokens)
print("Answer:", tokenizer.decode(predict_answer_tokens))

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.4907, -4.9752, -5.5313, -4.5826, -6.9588, -6.7018, -5.3166, -6.9251,
         -6.2860, -7.4585, -6.5807, -3.6467, -4.9902, 10.2982, -0.1105, -0.3647,
         -2.0145, -1.9959, -0.6065, -1.0672, -1.8010, -0.5006, -2.0730, -1.3307,
         -1.0903, -1.0370, -2.2645, -2.3040, -1.6029, -3.1273, -1.7220, -3.7256,
         -3.3881, -3.3806, -2.9219, -3.8486, -1.7546, -3.2455, -3.4626, -1.7635,
         -3.0164, -3.0037, -2.0180, -3.4991, -2.1505, -3.3018, -2.3693, -0.6510,
          1.0474, -2.6945, -1.7123, -3.5020, -2.9751, -2.1200, -0.3048, -0.8411,
         -1.1098, -2.0919, -1.0345, -2.0249, -1.5808, -2.9544, -3.5332, -4.6493,
         -3.3474, -4.5280, -2.9004, -2.2830, -3.2320, -4.0790, -2.6142, -1.0189,
         -4.3419, -3.1688, -1.7258, -0.7010, -4.1642, -1.8776, -5.1914, -5.0701,
         -1.9607, -1.5386, -2.8401, -3.8825, -3.9653, -1.1917, -1.3255, -2.7823,
         -4.5489, -2.2380, -1.2386, -4.2996, -1.2762, -4